In [33]:
import pandas as pd
import scipy.io as sio
import numpy as np
import random

data = sio.loadmat("secuencias.mat")

data


{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon May 28 11:54:56 2018',
 '__version__': '1.0',
 '__globals__': [],
 'sec_decodificacion': array([[array([[(array([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
                         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                         1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                         2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                     

In [34]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'sec_decodificacion', 'sec_aprendizaje'])

- $\epsilon$ el conjunto de estados posibles
- $X$ conjunto de símbolos de salida posibles
- $x_i$ observación en el instante $i$
- $L$ el largo de la secuencia
- $b_e (x)$ la probabilidad de emisión para la salida $x \epsilon X$ en el estado $e \epsilon \epsilon$
- $a_{ij}$ probabilidad de transición del estado $i$ al estado $j$
- $s^*=[s_1^*,...,s_L^*]$ secuencia de estados más probables

In [35]:
def viterbi(obs, prob_emision, prob_inicio, prob_transicion, estados):
    L = len(obs)
    N = len(estados)

    # Inicialización de matrices gamma y psi
    gamma = [[prob_emision[estados[i]][obs[0]] * prob_inicio[estados[i]] for i in range(N)]]
    psi = [[0 for _ in range(N)]]

    # Iteración a través de la secuencia de observaciones
    for k in range(1, L):
        gamma_app = []
        psi_app = []
        for i, e in enumerate(estados):
            max_val, max_state = max((gamma[k - 1][j] * prob_transicion[estados[j]][e], j) for j in range(N))
            gamma_app.append(prob_emision[e][obs[k]] * max_val)
            psi_app.append(max_state)
        
        gamma.append(gamma_app)
        psi.append(psi_app)

    # Encontrar el valor máximo en la última columna de gamma
    p_star, s_star_L = max((gamma[-1][i], i) for i in range(N))

    # Inicializar la secuencia de estados más probable
    s_star = [0] * L
    s_star[L - 1] = s_star_L

    # Retroceso para encontrar la secuencia de estados más probable
    for k in range(L - 1, 0, -1):
        s_star[k - 1] = psi[k][s_star[k]]

    # Mapear índices de estados a nombres de estados
    s_star = [estados[i] for i in s_star]

    return p_star, s_star


In [36]:
cod_estados = {
	1 : "coding",
	2 : "non-coding"
}

cod_bases = {
	1 : "A",
	2 : "T",
	3 : "C",
	4 : "G"
}

In [37]:
sec_aprendizaje = data["sec_aprendizaje"]

# Inicializamos listas para almacenar los datos
estados_list = []
salidas_list = []

# Iteramos a través del array de estructuras y extraemos los datos
for item in sec_aprendizaje[0]:
    for sub_item in item:
        estados = sub_item['estados'][0][0]
        salidas = sub_item['salidas'][0][0]
        
        # Convertimos los arrays a listas y los agregamos a las listas principales
        estados_list.append(estados)
        salidas_list.append(salidas)

# Convertimos las listas a arrays de numpy
estados_array = np.array(estados_list)
transformacion = np.vectorize(lambda x: cod_estados[x])
estados_array = transformacion(estados_array)
salidas_array = np.array(salidas_list)
transformacion = np.vectorize(lambda x: cod_bases[x])
salidas_array = transformacion(salidas_array)


In [38]:
estados_array

array([['non-coding', 'non-coding', 'non-coding', ..., 'coding',
        'coding', 'coding'],
       ['non-coding', 'non-coding', 'non-coding', ..., 'coding',
        'coding', 'coding'],
       ['non-coding', 'non-coding', 'non-coding', ..., 'non-coding',
        'non-coding', 'non-coding'],
       ['non-coding', 'non-coding', 'non-coding', ..., 'non-coding',
        'non-coding', 'non-coding']], dtype='<U10')

In [39]:
estados = ["coding", "non-coding"]
prob_emision = {
	"coding" : {
		"A" : 30/100,
		"T" : 30/100,
		"C" : 20/100,
		"G" : 20/100
	},
	"non-coding" : {
		"A" : 10/100,
		"T" : 10/100,
		"C" : 40/100,
		"G" : 40/100
	}
}

prob_transicion = {
	"coding" : {
		"coding" : 99.5/100,
		"non-coding" : 0.5/100
	},
	"non-coding" : {
		"coding" : 1/100,
		"non-coding" : 99/100
	}
}

prob_inicio = {
	"non-coding" : 0,
	"coding" : 0
}


for item in estados_array:
	prob_inicio[item[0]] += 1

prob_inicio["non-coding"] = prob_inicio["non-coding"]/4
prob_inicio["coding"] = prob_inicio["coding"]/4

prob_inicio

# Llamada al algoritmo de Viterbi
for item in salidas_array:
	p_star, s_star = viterbi(item, prob_emision, prob_inicio, prob_transicion, estados)
	print(f"Probabilidad de ocurrencia: {p_star}")
	print(f"Secuencia de estados más probable: {s_star}")

Probabilidad de ocurrencia: 0.0
Secuencia de estados más probable: ['non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-coding', 'non-cod

## Estimación de parámetros

In [ ]:
estados = ["coding", "non-coding"]
prob_emision = {
	"coding" : {
		"A" : random.random(),
		"T" : random.random(),
		"C" : random.random(),
		"G" : random.random()
	},
	"non-coding" : {
		"A" : random.random(),
		"T" : random.random(),
		"C" : random.random(),
		"G" : random.random()
	}
}

prob_transicion = {
	"coding" : {
		"coding" : random.random(),
		"non-coding" : random.random()
	},
	"non-coding" : {
		"coding" : random.random(),
		"non-coding" : random.random()
	}
}

for state in prob_emision:
    total = sum(prob_emision[state].values())
    for symbol in prob_emision[state]:
        prob_emision[state][symbol] /= total

for state in prob_transicion:
    total = sum(prob_transicion[state].values())
    for next_state in prob_transicion[state]:
        prob_transicion[state][next_state] /= total

prob_inicio = {
    "coding": 0.5, 
    "non-coding": 0.5
    }